In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [3]:
URM_test=sps.load_npz("../dataset/processed_data/URM_test_leaderboard.npz")

In [4]:
URM_test.shape

(4439758, 23692)

In [5]:
np.ravel(URM_test.sum(axis=1)).shape

(4439758,)

In [6]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [7]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [8]:
is_sorted(Session_ids)

True

In [9]:
dataset=None
dataset_max=None
#["UCF","ICBF","Graph","ICF","UCF_W","ICBF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW"]
selected_list = ["UCF_W","ICBF_W","Graph_W","ICF_W"]
for file in selected_list:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"../dataset/candidates/traditional_recs/leaderboard/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading UCF_W
16338555
loaded UCF_W
0
loading ICBF_W
14870776
loaded ICBF_W
0
loading Graph_W
8315730
loaded Graph_W
0
loading ICF_W
17772188
loaded ICF_W


In [10]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids

In [11]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [12]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [14]:
#import ast
#submission_df=pd.read_csv("../dataset/candidates/NN/GRU/leaderboard/candidates.csv")
#submission_df["score"]=submission_df["score"].map(ast.literal_eval)
#submission_df["code"]=submission_df["code"].map(ast.literal_eval)
#submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
#submission_df = submission_df.explode(['code','score', 'rank'])
#submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
#submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [15]:
import ast
submission_df_tr=pd.read_csv("../dataset/candidates/NN/Transformer/leaderboard/candidates.csv")
submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [16]:
#import ast
#submission_df_LSTM=pd.read_csv("../dataset/candidates/NN/LSTM/leaderboard/candidates.csv")
#submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
#submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
#submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
#submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
#submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
#submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [18]:
#dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
#del submission_df
dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
del submission_df_tr
#dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [19]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [20]:
dataset.fillna(0,inplace=True)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_39157/2573695804.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.fillna(0,inplace=True)


In [21]:
dataset_macro_features = pd.read_csv('../dataset/processed_data/macro_feats_NN_leaderboard.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items'])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [22]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [23]:
dataset.head()

,Item_ID,Score_UCF_W,Score_ICBF_W,Score_Graph_W,Score_ICF_W,Session_Id,score_transformer,rank_transformer,Max_Score_UCF_W,Max_Score_ICBF_W,...,Max_Score_ICF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,3,0.000000,0.000000,0.010701,1.350410e-02,26,0.000000,0.0,54.82594,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,9,0.000000,0.491950,0.000000,1.882741e-22,26,0.000000,0.0,54.82594,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,19,0.000000,0.183125,0.000000,0.000000e+00,26,0.000000,0.0,54.82594,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,21,0.095758,0.000000,0.019740,0.000000e+00,26,0.002732,43.0,54.82594,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,33,0.000000,0.730614,0.000000,6.852990e-03,26,0.000000,0.0,54.82594,3.397799,...,0.330278,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [24]:
gc.collect()

1587

In [25]:
dataset=dataset.sort_values("Session_Id")

In [26]:
gc.collect()

0

In [27]:
import lightgbm

In [28]:
k=10

In [29]:
result = []

In [30]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'../rankers/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))
    # dataset[f"score_{index}"]=model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1))

#combine scores
result = np.array(result)
result

array([[-0.90917261, -1.45082936, -1.45082936, ..., -1.05403352,
        -1.29830724, -0.19505178],
       [-2.49905718, -3.52995355, -3.57484424, ..., -2.40960111,
        -2.87277709, -0.96055199],
       [-2.5816811 , -3.54534641, -3.57164411, ..., -2.52743482,
        -2.69550466, -1.20536501],
       ...,
       [-2.66606892, -3.74370527, -3.76266467, ..., -2.59165316,
        -2.86017399, -1.13683523],
       [-2.55463509, -3.80685266, -3.7901756 , ..., -2.55961629,
        -2.64007414, -1.08682826],
       [-2.52289299, -3.55075361, -3.66605732, ..., -2.49538173,
        -2.69586995, -1.06302466]])

In [32]:
result.shape

(10, 30543091)

In [33]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [34]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

49

In [35]:
dataset.head()

,Session_Id,Item_ID,score
0,26,3,-2.348486
390,26,3373,-3.397643
391,26,3374,-3.413051
392,26,3375,-3.413051
393,26,3376,-3.413051


In [36]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [37]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [38]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [39]:
dataset.iloc[0]

Session_Id                                                   26
Item_ID       [3, 3373, 3374, 3375, 3376, 3378, 3379, 3372, ...
score         [-2.3484861037040936, -3.3976431972354453, -3....
indices       [424, 299, 206, 350, 455, 202, 164, 355, 200, ...
Name: 0, dtype: object

In [40]:
dataset.iloc[0]["Item_ID"][536]

1805

In [41]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [42]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_39157/744460501.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  items=np.array(x[0])
/var/folders/yt/h_zrk_k931s4wj_999xrfy0m0000gq/T/ipykernel_39157/744460501.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return items[x[1]]


In [43]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [45]:
dataset

,Session_Id,predictions,rank
0,26,"[606, 1008, 4685, 1337, 2905, 4881, 4462, 1205...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[1827, 772, 3893, 3400, 270, 4878, 3656, 1449,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[2289, 3398, 4986, 1181, 2235, 1039, 2075, 914...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[2378, 3337, 3937, 2369, 4155, 794, 1633, 4686...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[1004, 2175, 3586, 3172, 25, 1518, 1165, 2485,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[2258, 4387, 4810, 3526, 2605, 3867, 3084, 378...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[1875, 2877, 3880, 3026, 3153, 3155, 2216, 111...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[1901, 4779, 2638, 4846, 2431, 625, 1780, 3487...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[3531, 42, 1015, 989, 360, 2266, 2696, 3665, 1...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [46]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [47]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [48]:
dataset.columns=["session_id","code","rank"]

In [49]:
dataset.head()

,session_id,code,rank
0,26,"(606, 1008, 4685, 1337, 2905, 4881, 4462, 1205...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(1827, 772, 3893, 3400, 270, 4878, 3656, 1449,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(2289, 3398, 4986, 1181, 2235, 1039, 2075, 914...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(2378, 3337, 3937, 2369, 4155, 794, 1633, 4686...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(1004, 2175, 3586, 3172, 25, 1518, 1165, 2485,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [50]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [51]:
mapping= pd.read_csv('../dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [52]:
print(len(submission_df))

5000000


In [53]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 114.4 MB


In [54]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [55]:
submission_df.to_csv(
    f'../leaderboard_sub.csv',
    index=False
    )